# Includes

In [1]:
import mujoco
import sys
import os
import math
from tqdm import tqdm
import mediapy as media



# Classes and functions

In [2]:
class UR5e:
  
    def __init__(self, path):
        print('UR5e init.')
        self.m = mujoco.MjModel.from_xml_path(path)
        self.d = mujoco.MjData(self.m)
        self.startPos = [0, 0, 0, -math.pi/2, -math.pi/2, 0]
        for i in range(50):
            self.startPos.append(0)
        self.d.qpos = self.startPos
        mujoco.mj_forward(self.m, self.d)
    
    def __del__(self):
        print('UR5e del.')

In [3]:
def mycontroller(m, d):
    qacc = d.qacc
    for i in range(56):
        qacc[i] = 0
    
    d.qacc = qacc
    mujoco.mj_inverse(m, d)
    for i in range(6):
        d.ctrl[i] = d.qfrc_inverse[i]

In [4]:
class DataSaver:
    def __init__(self):
        self.data = []
    
    def add_data(self, d):
        self.data.append(d.qpos[0:56])

    def save_to_file(self, filename):
        with open(filename, 'w') as f:
            for d in self.data:
                f.write(','.join([str(x) for x in d]))
                f.write('\n')

# Simulation

In [5]:
xml_path = '../Models/universal_robots_ur5e/scene.xml'
duration = 5
max_steps = int(duration/0.002)
recordVideo = True
framerate = 20

In [6]:
if 'ur5e' in locals():
    print('ur5e already exists, deleting it')
    del ur5e

try:
    ur5e
except NameError:
    print('ur5e does not exist, creating it')
    ur5e = UR5e(xml_path)


ur5e does not exist, creating it
UR5e init.


In [7]:
if 'renderer' in locals():
    print('renderer already exists')
    del renderer


try:
    renderer
except NameError:
    renderer = mujoco.Renderer(ur5e.m, 480, 640)

In [8]:
stepsize = ur5e.m.opt.timestep
mujoco.set_mjcb_control(mycontroller)
ds = DataSaver()

In [9]:

idx = 0
frames = []
progress_bar = tqdm(total=duration)
while ur5e.d.time < duration and idx < max_steps:
    progress_bar.update(stepsize)
    #progress_bar.set_description(f"Time: {ur5e.d.time:.2f}")

    ds.add_data(ur5e.d)
    
    if len(frames) < ur5e.d.time * framerate and recordVideo:
        renderer.update_scene(ur5e.d, camera="fixed", scene_option=dict())
        pixels = renderer.render()
        frames.append(pixels)

    mujoco.mj_step(ur5e.m, ur5e.d)
    idx += 1
    

 80%|████████  | 4.003999999999781/5 [00:00<00:00, 10.57it/s] 

In [10]:
if recordVideo:
    media.show_video(frames, fps=framerate)
    
ds.save_to_file('data.csv')

100%|█████████▉| 4.999999999999671/5 [00:19<00:00, 10.57it/s]